## Importing required libraries

In [27]:
import numpy as np
import datetime
import os
import random, shutil
import glob

import warnings
warnings.simplefilter('ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

from matplotlib import pyplot as plt
import seaborn as sns
from matplotlib import pyplot
from matplotlib.image import imread

from os import makedirs,listdir
from shutil import copyfile
from random import seed
from random import random
import keras 
import tensorflow as tf
print(tf.__version__)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D, Input
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import InceptionV3
from sklearn import metrics


sns.set_context('notebook')
sns.set_style('white')

2.13.0


In [28]:
import gdown
debris_data="https://drive.google.com/file/d/1d6PxFpCPbAQJPuo7yz5dCohUxXQ1rZvR/view?usp=sharing"
output_path='nasa_images.zip'
gdown.download(debris_data, output_path, quiet=False,fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1d6PxFpCPbAQJPuo7yz5dCohUxXQ1rZvR
To: e:\TECH\ML\2024\Ocean_Challenge_Code\nasa_images.zip
100%|██████████| 4.12M/4.12M [00:08<00:00, 464kB/s]


'nasa_images.zip'

In [29]:
from zipfile import ZipFile
with ZipFile("nasa_images.zip", 'r') as zip:
 # printing all the contents of the zip file
  zip.printdir()
# extracting all the files in current working directory 
  zip.extractall()

File Name                                             Modified             Size
nasa_images/test/                              2024-01-03 14:59:24            0
nasa_images/test/image_jpeg(137).jpg           2021-10-28 09:39:40         7485
nasa_images/test/image_jpeg(90).jpg            2021-10-28 09:39:48         6613
nasa_images/train/                             2024-01-03 14:59:54            0
nasa_images/train/debris/                      2024-01-03 14:59:24            0
nasa_images/train/debris/1.jpg                 2021-10-28 09:39:48         7025
nasa_images/train/debris/image_jpeg (1).jpg    2021-10-28 09:40:06         6076
nasa_images/train/debris/image_jpeg (10).jpg   2021-10-28 09:39:26         6163
nasa_images/train/debris/image_jpeg (100).jpg  2021-10-28 09:39:34         6312
nasa_images/train/debris/image_jpeg (101).jpg  2021-10-28 09:40:12         7368
nasa_images/train/debris/image_jpeg (102).jpg  2021-10-28 09:40:00         6804
nasa_images/train/debris/image_jpeg (103

In [32]:
img_rows, img_cols = 150, 150
batch_size = 32
n_epochs = 10
n_classes = 2
val_split = 0.2
verbosity = 1
path = 'nasa_images/train/'
path_test = 'nasa_images/test/'
input_shape = (img_rows, img_cols, 3) #RGB
labels = ['debris', 'no_debris']
seed = 10
checkpoint_path='OceanNet.h5'

In [33]:
# Create ImageDataGenerators for training and validation and testing
train_datagen = ImageDataGenerator(
    validation_split = val_split,
    rescale=1.0/255.0,
	width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    validation_split = val_split,
    rescale=1.0/255.0,
	width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255.0
)

In [ ]:
# use the labels defined before to 
# find number of images belonging to each category
train_generator = train_datagen.flow_from_directory(
    directory = path,
    classes = labels,
    seed = seed,
    batch_size = batch_size, 
    class_mode='binary',
    shuffle = True,
    target_size=(img_rows, img_rows),
    subset = 'training'
)

In [ ]:
val_generator = val_datagen.flow_from_directory(
    directory = path,
    classes = labels,
    seed = seed,
    batch_size = batch_size, 
    class_mode='binary',
    shuffle = True,
    target_size=(img_rows, img_rows),
    subset = 'validation'
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    directory = path_test,
    classes = labels,
    class_mode='binary',
    seed = seed,
    batch_size = batch_size, 
    shuffle = True,
    target_size=(img_rows, img_rows)
)